# Two-dimensional Kakeya needle problem



In [ ]:
# @title Prepare dependencies

!pip install shapely

In [ ]:
# @title Utils

import time
from collections.abc import Mapping
from typing import Any

from matplotlib import pyplot as plt
from matplotlib.patches import Polygon as mpl_Polygon
import matplotlib.pyplot as plt
import numpy as np
from shapely import geometry


def plot_polygons(polygons_vertices: list[Any], title='Triangles'):
  """Plots a list of polygons using matplotlib.

  Args: polygons_vertices : A list of polygon vertices.
      title (str, optional): Title of the plot. Defaults to "Collection of
      Parallelograms".
  """
  fig, ax = plt.subplots(figsize=(12, 8))
  polygons = list()

  # Convert list of vertices to shapely polygons
  for polygon_vertices in polygons_vertices:
    polygon = geometry.Polygon(polygon_vertices)
    polygons.append(polygon)

  for polygon in polygons:
    # Get the exterior coordinates of the polygon
    exterior_coords = list(polygon.exterior.coords)
    # Create a matplotlib.patches.Polygon patch
    patch = mpl_Polygon(
        np.array(exterior_coords),
        closed=True,
        facecolor='lightblue',
        edgecolor='blue',
        linewidth=1,
        alpha=0.7,
    )
    ax.add_patch(patch)

    # Optionally plot interiors (holes) if the polygon has them
    for interior in polygon.interiors:
      interior_coords = list(interior.coords)
      interior_patch = mpl_Polygon(
          np.array(interior_coords),
          closed=True,
          facecolor='white',
          edgecolor='blue',
          linewidth=1,
          alpha=0.7,
      )
      ax.add_patch(interior_patch)

  # Adjust plot limits to fit all parallelograms
  bounds = [p.bounds for p in polygons]
  minx = min([b[0] for b in bounds])
  miny = min([b[1] for b in bounds])
  maxx = max([b[2] for b in bounds])
  maxy = max([b[3] for b in bounds])
  ax.set_xlim(minx - 0.1, maxx + 0.1)  # Add some padding
  ax.set_ylim(miny - 0.1, maxy + 0.1)
  ax.set_aspect('equal', adjustable='box')  # Ensure equal aspect ratio
  ax.set_title(title, fontsize=15)
  plt.xlabel('X', fontsize=15)
  plt.ylabel('Y', fontsize=15)
  plt.grid(True)
  plt.show()

**Prompt for the AlphaEvolve search setup**

Act as a research mathematician, software developer and optimization specialist
specializing in constructing lists of floats with certain extremal properties.

GOAL:
Your task is to find an algorithm which, for a given
natural number n > 1, produces a python list x = (x_i)_(i=1)^n consisting of
n positive floats ("positions") which optimize the following task.

Let us define $a_i = i / n$ for $i = 1, \dots, n$.
For each $i = 0, \dots, n-1$ let us consider the two-dimensional triangle $R_i$
in the plane given by vertices $(x_i, 0), (x_i + 1 / n, 0), (x_i + a_i, 1)$.

Find x so that the union of the triangles $R_i$ has smallest possible area.

Specifically, the Python function you have to provide is called get_positions.
For example, here is an starting implementation to get you going:

def get_positions(n: int) -> np.ndarray:
  """Generate positions to minimize the area of the union."""
  variable_name = "suggested_positions_" + str(n)
  best_positions = np.linspace(0, 1, n)

  if np.random.rand() < 0.5 and variable_name in globals():
    best_positions = globals()[variable_name]
  curr_positions = best_positions.copy()
  best_score = get_score(best_positions)
  start_time = time.time()
  while time.time() - start_time < 100:
    random_index = np.random.randint(0, len(curr_positions))
    curr_positions[random_index:] += 1e-1 * np.random.randint(-10, 10)
    curr_score = get_score(curr_positions)
    if curr_score > best_score:
      best_score = curr_score
      best_positions = curr_positions.copy()
  return best_positions

The function get_positions applies your algorithm and outputs the endpoints
[x_1, x_2, ... ,x_n] of the positions.

EVALUATION:
Your proposed construction will be evaluated by an external function that
computes the area of the union and negates it, i.e. it takes your list x
as an input and produces the negative total union area as a float
get_score(x). A higher score (i.e. lower area) is better.

You don't have to implement this scoring function; it's already found elsewhere
in the codebase.

HINTS:
The previous solution provided in this prompt is not optimal, much
better algorithms are possible. The patterns you have to discover are not
hard, you can definitely improve it, it is not beyond your capabilities. DO NOT
go for the same solution as the previous one. Always try to find a better
pattern, don't be scared of the difficult sounding problem, once you see the
solution you'll realise it wasn't hard at all. Good luck, I believe in you, but
you also have to believe in yourself!

You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have access to it,
you don't need to code up the get_score() function.
You want the score it gives you to be as high as possible!

Your task is to write a search function that searches for the best list.
Your function will have 1000 seconds to run, and after that it has to have
returned the best construction it found. If after 1000 seconds it has not
returned anything, it will be terminated with negative infinity points. You can
use your time best if you have an outer loop of the form
"while time.time() - start_time < 1000:" or similar, just don't forget to define
the "start_time" variable early in your program. You'll have to balance your
resources carefully!

In [ ]:
# @title Evaluation for the AlphaEvolve search setup

import numpy as np
import enum
import dataclasses

class ErrorMessage(enum.Enum):
  NO_ERROR = 0
  WRONG_OUTPUT_SHAPE = 1
  FUNCTION_IS_TOO_LARGE = 2
  ZERO_FUNCTION_SCORE = 3
  STOCHASTIC_FUNCTION = 4
  OTHER_ERROR = 5
  NO_POLYGON_SET = 6
  POINTS_NOT_FLOATS = 7


@dataclasses.dataclass
class AlgorithmEvaluationLog:
  scores: list[float]
  positions: dict[str, Any]
  error_messages: list[ErrorMessage]


@dataclasses.dataclass(frozen=True)
class AreaEvaluationLog:
  area: float
  error_message: ErrorMessage


def cross_product_z(p1, p2, p3):
  """Calculates the Z component of the cross product (p2-p1) x (p3-p1).

  Determines the orientation of p3 relative to the vector p1->p2.
  """
  return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])


def is_point_in_triangle(point, triangle_vertices):
  """Checks if a point is inside a triangle using the cross-product method.

  Includes points on the boundary. Assumes counter-clockwise or clockwise vertex
  order.
  """
  v1, v2, v3 = triangle_vertices

  # Check orientation of the point relative to each edge using cross product sign
  d1 = cross_product_z(v1, v2, point)
  d2 = cross_product_z(v2, v3, point)
  d3 = cross_product_z(v3, v1, point)

  # Check if the point is on the same side (or on the line) of all edges
  has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
  has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)

  # Point is inside or on edge if signs are all non-negative or all non-positive
  return not (has_neg and has_pos)


def get_all_triangle_vertices(x: np.ndarray) -> list[Any]:
  """Generates the list of triangle vertices for a triangle construction."""
  n = len(x)
  delta = 1 / n
  triangle_vertex_list = list()
  for i in range(1, n + 1):
    ai = i / n
    xi = x[i - 1]
    if not isinstance(xi, float):
      return AreaEvaluationLog(
          area=0.0,
          error_message=ErrorMessage.POINTS_NOT_FLOATS,
      )
    vertices = [
        (xi, 0),
        (xi + delta, 0),
        (xi + ai, 1),
    ]
    triangle_vertex_list.append(vertices)
  return triangle_vertex_list


def get_all_parallelogram_vertices(x: np.ndarray) -> list[Any]:
  """Generates the list of triangle vertices of a parallelogram construction."""
  n = len(x)
  delta = 1 / n
  triangle_vertex_list = []
  for i in range(1, n + 1):
    ai = i / n
    xi = x[i - 1]
    if not isinstance(xi, float):
      return AreaEvaluationLog(
          area=0.0,
          error_message=ErrorMessage.POINTS_NOT_FLOATS,
      )

    # For numerical stability, we create two triangles,
    # one for the lower part of the parallelogram and one for the upper part.
    vertices_lower_triangle = [
        (xi, 0),
        (xi + delta, 0),
        (xi + ai, 1),
    ]
    vertices_upper_triangle = [
        (xi + delta, 0),
        (xi + ai + delta, 1),
        (xi + ai, 1),
    ]
    triangle_vertex_list.append(vertices_lower_triangle)
    triangle_vertex_list.append(vertices_upper_triangle)
  return triangle_vertex_list


def get_score_no_shapely(triangles: list[Any], grid_resolution=1000):
  """Calculates the approximate area of the union of triangles.

  using grid sampling (without Shapely).

  Args:
    x: NumPy array of length n, defining the base x-coordinates.
    grid_resolution: Number of sampling points along the longest dimension.
      Higher values increase accuracy and computation time.

  Returns:
    An approximation of the union area, or FUNCTION_IS_TOO_LARGE.
  """
  n = len(triangles)

  # Handle n=0 case explicitly
  if n == 0:
    return 0.0

  all_vertices = list()

  for triangle in triangles:
    all_vertices.extend(triangle)

  # Find Bounding Box of all triangles
  # Y bounds are fixed by the problem definition
  min_y = 0.0
  max_y = 1.0
  # X bounds depend on the input x and derived values
  min_x = min(v[0] for v in all_vertices)
  max_x = max(v[0] for v in all_vertices)

  width = max_x - min_x
  height = max_y - min_y  # This will be 1.0

  # Handle degenerate cases where the bounding box has zero area
  # Use a small tolerance for floating point comparisons
  if width <= 1e-9 or height <= 1e-9:
    return 0.0

  # 4. Grid Setup
  # Use the same resolution count for both axes for simplicity.
  # Cell dimensions depend on bounding box aspect ratio.
  step_x = width / grid_resolution
  step_y = height / grid_resolution  # height is 1.0
  cell_area = step_x * step_y

  # 5. Grid Iteration & Point-in-Union Check
  inside_points_count = 0
  for i in range(grid_resolution):
    # Sample point at the center of the grid cell
    px = min_x + (i + 0.5) * step_x
    for j in range(grid_resolution):
      py = min_y + (j + 0.5) * step_y
      point = (px, py)

      # Check if this point is inside *any* of the generated triangles
      point_is_in_union = False
      for triangle in triangles:
        if is_point_in_triangle(point, triangle):
          point_is_in_union = True
          break  # Point is in the union, no need to check other triangles

      if point_is_in_union:
        inside_points_count += 1

  # 6. Calculate Approximate Area
  approx_area = inside_points_count * cell_area

  return approx_area


def get_score_shapely(triangles: list[Any]) -> AreaEvaluationLog:
  """Computes the area of a given configuration of triangles using shapely."""

  if not triangles:
    return -100000

  polygons = list()
  for triangle in triangles:
    polygons.append(geometry.Polygon(triangle))

  # Calculate the union of all triangles
  union_polygon = polygons[0]
  for i in range(1, len(polygons)):
    union_polygon = union_polygon.union(polygons[i])

  return union_polygon.area

In [ ]:
# @title Keich construction for triangles

def get_keich_positions(n: int) -> np.ndarray:
  pos = int(np.log2(n))
  j = np.arange(1, pos + 1)
  x = list()
  for i in range(n):
    eps = np.array(list(str(bin(i))[2:]), dtype=int)
    eps = np.pad(eps, (pos - len(eps), 0), 'constant')
    x.append(np.sum((1 - j) * eps * (2.0 ** -j) / pos))
  return x

keich_positions = get_keich_positions(2**2)

# keich_positions
keich_triangles = get_all_triangle_vertices(get_keich_positions(2**2))
plot_polygons(keich_triangles, title="Keich Construction")

In [ ]:
# @title Collect Keich triangles scores
keich_triangle_shapely_scores = list()
keich_triangle_no_shapely_scores = list()


for n in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
  keich_positions = get_keich_positions(2**n)
  keich_triangles = get_all_triangle_vertices(get_keich_positions(2**n))
  keich_triangle_shapely_score = get_score_shapely(keich_triangles)
  keich_triangle_shapely_scores.append(keich_triangle_shapely_score)

In [ ]:
# @title Keich construction for parallelograms

def get_keich_positions(n: int) -> np.ndarray:
  pos = int(np.log2(n))
  j = np.arange(1, pos + 1)
  x = list()
  for i in range(n):
    eps = np.array(list(str(bin(i))[2:]), dtype=int)
    eps = np.pad(eps, (pos - len(eps), 0), "constant")
    x.append(np.sum((1 - j) * eps * (2.0**-j) / pos))
  return x


keich_positions = get_keich_positions(2**6)

# keich_positions
keich_parallelograms = get_all_parallelogram_vertices(get_keich_positions(2**6))
plot_polygons(keich_parallelograms, title="Keich Construction")

In [ ]:
# @title Collect Keich parallelogram scores

keich_parallelogram_scores = dict()

for n in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
  keich_positions = get_keich_positions(2**n)
  keich_parallelograms = get_all_parallelogram_vertices(get_keich_positions(2**n))
  keich_parallelogram_score = get_score_shapely(keich_parallelograms)
  keich_parallelogram_scores[2**n] = keich_parallelogram_score

## AE Constructions

In [ ]:
# @title AE Triangle constructions

def get_positions_area(x: np.ndarray | list[float]) -> AreaEvaluationLog:
  """Computes the area of a given configuration of triangles."""
  n = len(x)

  delta = 1 / n
  triangles = []
  for i in range(1, n + 1):
    ai = i / n
    xi = x[i - 1]
    if not isinstance(xi, float):
      return AreaEvaluationLog(
          area=0.0,
          error_message=ErrorMessage.POINTS_NOT_FLOATS,
      )

    vertices = [
        (xi, 0),
        (xi + delta, 0),
        (xi + ai, 1),
    ]
    triangles.append(geometry.Polygon(vertices))

  if not triangles:
    return AreaEvaluationLog(
        area=0.0,
        error_message=ErrorMessage.NO_POLYGON_SET,
    )

  # Calculate the union of all triangles
  union_polygon = triangles[0]
  for i in range(1, len(triangles)):
    union_polygon = union_polygon.union(triangles[i])

  return AreaEvaluationLog(
      area=union_polygon.area,
      error_message=ErrorMessage.NO_ERROR,
  )

ae_triangle_constructions = []

suggested_positions_2 = [511.99816177, 511.8314951]
ae_triangle_constructions.append(suggested_positions_2)
print("For N = 2: ", "AE: ", get_positions_area(suggested_positions_2).area, "Keich: 0.375")

suggested_positions_4 = [541.64802077, 541.52307854, 541.52296299, 541.39802077]
ae_triangle_constructions.append(suggested_positions_4)
print("For N = 4: ", "AE: ", get_positions_area(suggested_positions_4).area, "Keich: 0.2734375")

suggested_positions_8 = [0.15358956, 0.08446718, 0.10888746, 0.03490196, -0.03908353, -0.00786618, -0.09133793, -0.11512768]
ae_triangle_constructions.append(suggested_positions_8)
print("For N = 8: ", "AE: ", get_positions_area(suggested_positions_8).area, "Keich: 0.21788194444444442")

suggested_positions_16 = [20.60335303, 20.56919091, 20.53502879, 20.57678627, 20.5333167 , 20.51281489, 20.46540799, 20.43506285, 20.50871178, 20.47836664, 20.43095973, 20.41045792, 20.36698835, 20.40874584, 20.37458371, 20.34042159]
ae_triangle_constructions.append(suggested_positions_16)
print("For N = 16: ", "AE: ", get_positions_area(suggested_positions_16).area, "Keich: 0.1810302734375")

suggested_positions_32 = [0.2783951201159106 , 0.2599095754467311 , 0.2696152341181265 , 0.2474679916071266 , 0.23342405120062426 , 0.21110576789583535 , 0.2503913051657332 , 0.22780678816954197 , 0.2111946276348219 , 0.18938214918827562 , 0.19635922811755205 , 0.17488191415673 , 0.157355394930442 , 0.13412297872595616 , 0.1890372955955657 , 0.16517706927595427 , 0.14956312995856672 , 0.1250672575883866 , 0.11403313067695249 , 0.09100146988497741 , 0.10694050427770534 , 0.08601681569221568 , 0.06901354733120776 , 0.04774530628400415 , 0.09917785730177683 , 0.0778734006680343 , 0.07470227741745773 , 0.05089688760546796 , 0.03633940715872134 , 0.013545330030818769 , 0.023412402190728043 , 0.0045907115621472636]
ae_triangle_constructions.append(suggested_positions_32)
print("For N = 32: ", "AE: ", get_positions_area(suggested_positions_32).area, "Keich: 0.15458984374999993")

suggested_positions_64 = [0.11383197, 0.10324642, 0.10146646, 0.09022036, 0.07901294, 0.08037822, 0.06914903, 0.05791487, 0.056083 , 0.04547135, 0.09510656, 0.08434572, 0.07362799, 0.07950383, 0.06738154, 0.062804 , 0.04969796, 0.03899234, 0.02593009, 0.01987598, 0.00751372, 0.00850378, -0.00273944, -0.0139662 , 0.01905029, 0.01059917, -0.00197299, -0.00772473, -0.02061307, -0.02634825, -0.03887649, -0.04731751, -0.03766288, -0.04759367, -0.05752164, 0.03428245, 0.02308072, 0.01704893, 0.00487998, -0.00337917, 0.01552485, 0.00416484, 0.00189282, -0.00911675, -0.02169696, -0.03269521, -0.02673608, -0.03864435, -0.05057074, -0.05582781, -0.06835264, -0.08088845, -0.088086 , -0.08768905, -0.09867628, -0.0558232 , -0.06695957, -0.07242037, -0.0848898 , -0.08997261, -0.10223612, -0.1101771 , -0.10309349, -0.11280547]
ae_triangle_constructions.append(suggested_positions_64)
print("For N = 64: ", "AE: ", get_positions_area(suggested_positions_64).area, "Keich: 0.13469441731770845")

suggested_positions_128 = [0.01356158, 0.00852387, 0.01041449, 0.00664847, 0.00030537, -0.00131449, -0.00644049, -0.01340839, -0.01709616, -0.01222271, -0.01751606, -0.02176912, -0.02673844, -0.00097485, -0.00635104, -0.01116838, -0.01169215, -0.01622249, -0.02262436, -0.02447167, -0.02947953, -0.01115498, -0.01730063, -0.02232313, -0.01802965, -0.02297936, -0.02973018, -0.03455186, -0.03969632, -0.04461961, -0.05034761, -0.04339253, -0.04846627, -0.05512463, -0.05949229, -0.06609674, -0.06561228, -0.07046129, -0.07585924, -0.04215149, -0.04753491, -0.05280425, -0.05255173, -0.05774912, -0.06348477, -0.06858414, -0.0747848 , -0.07155234, -0.07714036, -0.08062861, -0.08627907, -0.09106506, -0.06324889, -0.06879601, -0.07279415, -0.06225538, -0.06788587, -0.069356 , -0.07511017, -0.08060016, -0.08449615, -0.090853 , -0.0968939 , -0.10139471, -0.09893835, -0.10406425, -0.11022598, -0.11617424, -0.09421339, -0.09296197, -0.09678811, -0.10314564, -0.10886036, -0.10955369, -0.11501813, -0.12206476, -0.12797178, -0.13193593, -0.13388201, -0.14020283, -0.10565394, -0.11086007, -0.10900186, -0.11257522, -0.11834878, -0.1246185 , -0.12593551, -0.13266081, -0.13818461, -0.12823735, -0.13078755, -0.13613029, -0.14181128, -0.14764559, -0.14991815, -0.15624429, -0.16181564, -0.16654416, -0.15398468, -0.15911593, -0.16241838, -0.16850321, -0.17257323, -0.14009941, -0.14550102, -0.14024158, -0.14544472, -0.15114359, -0.15636165, -0.15340218, -0.15863782, -0.16531165, -0.16897461, -0.17528177, -0.17105793, -0.17641441, -0.18276781, -0.18545826, -0.1913079 , -0.17181909, -0.17546619, -0.1810009 , -0.18300711, -0.18859439, -0.19388462, -0.18888304, -0.19377542, -0.19843118]
ae_triangle_constructions.append(suggested_positions_128)
print("For N = 128: ", get_positions_area(suggested_positions_128).area, "Keich: 0.11920726542570147")

# Collect all AE triangle scores
ae_triangle_scores = []
for construction in ae_triangle_constructions:
  ae_triangle_scores.append(get_positions_area(construction).area)



In [ ]:
# @title AE Parallelogram constructions

ae_parallelogram_scores = []

ae_para_2 = [3.417426624777884, 3.167426624777884]
ae_para_2_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_2))
ae_parallelogram_scores.append(ae_para_2_shapely_score)
print("AE score: ", ae_para_2_shapely_score, "Keich score: ", keich_parallelogram_scores[2])

ae_para_4 = [0.37799912241758804 , 0.2529991252454076 , 0.12799912524540766 , 0.0029991252454076513]
ae_para_4_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_4))
ae_parallelogram_scores.append(ae_para_4_shapely_score)
print("AE score: ", ae_para_4_shapely_score, "Keich score: ", keich_parallelogram_scores[4])

ae_para_8 = [4.3749999584537846e-01, 3.7499999584505783e-01, 3.1249999879618023e-01, 2.5000000016841306e-01, 1.8750000077771614e-01, 1.2500000228656247e-01, 6.2500002391489756e-02, 2.2865625509032767e-09]
ae_para_8_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_8))
ae_parallelogram_scores.append(ae_para_8_shapely_score)
print("AE score: ", ae_para_8_shapely_score, "Keich score: ", keich_parallelogram_scores[8])

ae_para_16 = [5.2460078144829214e-01, 4.7238634034695498e-01, 4.6365251231392351e-01, 3.5202559554295687e-01, 2.9223713536433349e-01, 2.6091671565398222e-01, 2.3327321495463949e-01, 2.0205912724031627e-01, 1.7085059229528315e-01, 1.3962785368777081e-01, 1.0841017991296895e-01, 7.7200611416680456e-02, 8.1376889476915265e-03, 4.3469045174547999e-03, 1.0000000000000001e-09, 1.0000000000000001e-09]
ae_para_16_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_16))
ae_parallelogram_scores.append(ae_para_16_shapely_score)
print("AE score: ", ae_para_16_shapely_score, "Keich score: ", keich_parallelogram_scores[16])

ae_para_32 = [4.8620460759667949e-01, 4.2508154102538070e-01, 3.9016254155406582e-01, 3.6957126713921062e-01, 3.4580295645943887e-01, 3.4883727860627534e-01, 3.3549347462425999e-01, 2.8561077239478155e-01, 2.7063396240725457e-01, 2.4020743650639428e-01, 2.2150400272670887e-01, 2.1588559628298656e-01, 2.1338744352779981e-01, 1.9030865858574844e-01, 1.6143534613470267e-01, 1.3684114986094159e-01, 1.9837006518943967e-01, 1.6783997315082175e-01, 1.4092376589124650e-01, 1.6227460194341758e-01, 1.4387053848166942e-01, 1.2875350683602627e-01, 1.0141773836767930e-01, 9.2536221222586187e-02, 6.6342129510756645e-02, 5.2007203015977152e-02, 3.6776791752239045e-02, 6.5393428198790243e-05, 0.0000000000000000e+00,6.5393428198790243e-05, 6.5393428198790243e-05, 6.5393428198790243e-05]
ae_para_32_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_32))
ae_parallelogram_scores.append(ae_para_32_shapely_score)
print("AE score: ", ae_para_32_shapely_score, "Keich score: ", keich_parallelogram_scores[32])

# ae_para_64 = [8.8694838038513724e-01, 8.8293461465044454e-01, 8.7161935833140003e-01, 8.6912065023581886e-01, 8.5920556416586047e-01, 8.5240645602133946e-01, 8.4689261232166435e-01, 8.4028625878780172e-01, 7.9219578800388879e-01, 7.8686759438658149e-01, 7.7942491577238093e-01, 7.7519725399213324e-01, 7.6570797365911547e-01, 7.6079157284219801e-01, 6.1349535974619718e-01, 6.0640638945226810e-01, 6.0048513459151120e-01, 5.1836332733583990e-01, 5.1585068547271828e-01, 5.1207618656887233e-01, 5.0492160263509422e-01, 5.0237449330185835e-01, 4.6074890183308898e-01, 4.5720062455910226e-01, 4.5307521786989485e-01, 4.4411611714457089e-01, 4.4411611558468844e-01, 4.3416641237398229e-01, 4.3341082406611214e-01, 3.1130932567213770e-01, 3.0514605360407759e-01, 3.0054702516874698e-01, 2.9451910363373240e-01, 2.8872227937162820e-01, 2.8547254512348252e-01, 2.8103440915807237e-01, 2.7528257258210431e-01, 1.9957605873700504e-01, 1.9748601165185936e-01, 1.9027170716527117e-01, 1.8609180139387990e-01, 1.8609179983399746e-01, 1.8007410663959231e-01, 1.7379778751682429e-01, 1.7379778595694184e-01, 1.6759157820712323e-01, 1.6479736713277685e-01, 7.8943257056792615e-02, 7.3107576998144991e-02, 7.3107575438262587e-02, 6.9009479864337489e-02, 6.3363101632330332e-02, 6.1997312426898585e-02, 5.7901306178231429e-02, 9.0000000000000012e-09, 8.0000000000000005e-09, 6.9999999999999998e-09, 6.0000000000000000e-09, 5.0000000000000001e-09, 4.0000000000000002e-09, 3.0000000000000004e-09, 2.0000000000000001e-09, 1.0000000000000001e-09, 1.0000000000000001e-09]
ae_para_64 = [0.9106133876309866 , 0.9063513222461946 , 0.9020362327244008 , 0.8977571748594148 , 0.8934438892357935 , 0.8892228361176998 , 0.8307367864817257 , 0.8271328746094002 , 0.8236203748983703 , 0.8201426190415 , 0.8163116232631904 , 0.8130147249521643 , 0.8093288950305997 , 0.8057124954185663 , 0.8020985002752661 , 0.7985737410363268 , 0.7950338048048029 , 0.7300051987005886 , 0.7267744242141373 , 0.7235235928233308 , 0.720111624368684 , 0.7170082021816941 , 0.7135953277710236 , 0.71044681093939 , 0.7070342563611015 , 0.7038405391083544 , 0.7005547277383201 , 0.6474018800010605 , 0.643294302090786 , 0.6390538832292897 , 0.6350306691038012 , 0.630891617452926 , 0.6266719411658279 , 0.622611913006528 , 0.6184873268803761 , 0.47275960736833245, 0.4687328211987251 , 0.46457916520138093, 0.4606814148523254 , 0.45655285477528584, 0.45236280590091915, 0.4484062936396358 , 0.444392346250983 , 0.44030501781456605, 0.3821602470140765 , 0.37819001384135076, 0.3744412989107077 , 0.3706552823699953 , 0.36622852629866187, 0.36227472448632037, 0.3589122919152267 , 0.3549286577672196 , 0.3505331982907258 , 0.3467096449635284 , 0.34303169934858607, 0.3389033371376654 , 0.3350833984144761 , 0.2676087330567171 , 0.2631808610332871 , 0.2588898121340522 , 0.25439858948102395, 0.250086825997281 , 0.24565130962135867, 0.24118695423425707]
ae_para_64_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_64))
ae_parallelogram_scores.append(ae_para_64_shapely_score)
print("AE score: ", ae_para_64_shapely_score, "Keich score: ", keich_parallelogram_scores[64])

ae_para_128 = [0.8055790210893758 , 0.8032143129010282 , 0.8022975092487757 , 0.7990824001896396 , 0.7986442500884856 , 0.7963471878166002 , 0.7942403909613889 , 0.7924987959501636 , 0.7789525914005548 , 0.7777177323834584 , 0.7749849231589836 , 0.7746567450885409 , 0.7722378418842822 , 0.770599210950063 , 0.70951702560639 , 0.707543336113615 , 0.7073917836687524 , 0.7060153759718105 , 0.7029481931875409 , 0.7022202162462178 , 0.7000124698739634 , 0.699585656068663 , 0.6976499801553983 , 0.6752264356112606 , 0.6733672251477859 , 0.6722559268413064 , 0.6700196928554233 , 0.669701904268045 , 0.6666265406244483 , 0.6666265396244483 , 0.663648547613497 , 0.662164833685728 , 0.6618618577069606 , 0.660405475795175 , 0.6574979761010454 , 0.6570926713337162 , 0.654918318034027 , 0.6336634707435014 , 0.632515244824908 , 0.630546810116752 , 0.6291550973325343 , 0.6276982242487712 , 0.6258971057574176 , 0.5173587763930695 , 0.515976672165278 , 0.514150823997408 , 0.512614402800003 , 0.5111637717600578 , 0.5096332670448409 , 0.4904676233788717 , 0.48884406170740674, 0.4875128103874892 , 0.48567141978102096, 0.4843260525614999 , 0.48332671570461033, 0.4809820301875122 , 0.4806458233122125 , 0.47779544493402526, 0.4777954439340252 , 0.4757185513932993 , 0.47427202441105565, 0.4414839668546904 , 0.4396721172781276 , 0.4393769895514666 , 0.4382164726289623 , 0.43494269277894737, 0.43352838773448293, 0.4333520867177554 , 0.432620490295201 , 0.4294130143181767 , 0.42860175281590585, 0.4267025972852251 , 0.426101044924336 , 0.4242861572657023 , 0.39527951221170093, 0.39341093195773236, 0.3930663376399151 , 0.39085861189275267, 0.39078961320025246, 0.38769471905122493, 0.3876947180512249 , 0.38590391057146756, 0.38454523094725723, 0.38315160366535583, 0.3676341256951384 , 0.36618915183260226, 0.36454695508942475, 0.3644315256649038 , 0.2539621705071848 , 0.2522987942874384 , 0.25002213656750133, 0.2496078328048002 , 0.24708371420666214, 0.2453992011246301 , 0.24386964071938494, 0.22050540021651324, 0.21923445209462192, 0.21733277561096198, 0.21586961184810166, 0.21413518295134706, 0.21381920040416302, 0.21237287569867186, 0.20970854888015114, 0.2083321853541696 , 0.20760828545860327, 0.2058308978088706 , 0.17141956325161176, 0.17116330997171206, 0.1679867650681354 , 0.16793555285043602, 0.16458652141129784, 0.16458652041129784, 0.16279637384268503, 0.15948866719627966, 0.15948866619627966, 0.15655819574639193, 0.1559943306574625 , 0.15394549952432637, 0.15176845333636285, 0.15009238892619903, 0.11133615274307905, 0.1096150571244645 , 0.10751455337816833, 0.1050906579646864 , 0.10290492671601631, 0.10070446144308949, 0.09824982700505415, 0.09628267564062204]
ae_para_128_shapely_score = get_score_shapely(
    get_all_parallelogram_vertices(ae_para_128))
ae_parallelogram_scores.append(ae_para_128_shapely_score)
# ae_para_128_no_shapely_score = get_score_no_shapely(
#     get_all_parallelogram_vertices(ae_para_128), grid_resolution=1000)
print("AE score: ", ae_para_128_shapely_score, "Keich score: ", keich_parallelogram_scores[128])




In [ ]:
# @title Plots of constructions
# Plot Keich Triangle

# keich_triangles = get_all_triangle_vertices(get_keich_positions(2**4))
# plot_polygons(keich_triangles, title="Keich Construction N = 16")

# keich_triangles = get_all_triangle_vertices(get_keich_positions(2**5))
# plot_polygons(keich_triangles, title="Keich Construction N = 32")

# keich_triangles = get_all_triangle_vertices(get_keich_positions(2**6))
# plot_polygons(keich_triangles, title="Keich Construction N = 64")


# Plot AE Triangle

# ae_tri_16_vertices = get_all_triangle_vertices(suggested_positions_16)
# plot_polygons(ae_tri_16_vertices, title="AE Construction N = 16")

# ae_tri_32_vertices = get_all_triangle_vertices(suggested_positions_32)
# plot_polygons(ae_tri_32_vertices, title="AE Construction N = 32")

# ae_tri_64_vertices = get_all_triangle_vertices(suggested_positions_64)
# plot_polygons(ae_tri_64_vertices, title="AE Construction N = 64")


# Plot AE Para

# ae_para_16_vertices = get_all_parallelogram_vertices(ae_para_16)
# plot_polygons(ae_para_16_vertices, title="AE Construction N = 16")

# ae_para_32_vertices = get_all_parallelogram_vertices(ae_para_32)
# plot_polygons(ae_para_32_vertices, title="AE Construction N = 32")

# ae_para_64_vertices = get_all_parallelogram_vertices(ae_para_64)
# plot_polygons(ae_para_64_vertices, title="AE Construction N = 64")


# Plot Keich Para

# keich_parallelograms = get_all_parallelogram_vertices(get_keich_positions(2**3))
# plot_polygons(keich_parallelograms, title="Keich Construction")

# keich_parallelograms = get_all_parallelogram_vertices(get_keich_positions(2**4))
# plot_polygons(keich_parallelograms, title="Keich Construction N = 16")

# keich_parallelograms = get_all_parallelogram_vertices(get_keich_positions(2**5))
# plot_polygons(keich_parallelograms, title="Keich Construction N = 32")

# keich_parallelograms = get_all_parallelogram_vertices(get_keich_positions(2**6))
# plot_polygons(keich_parallelograms, title="Keich Construction N = 64")

In [ ]:
# @title Area graphs (triangles)


xs = 2 ** np.arange(1, 8)

plt.figure(figsize=(12, 8))  # Adjust figure size as needed

# Plot Function 1
plt.plot(
    xs,
    np.array(ae_triangle_scores),
    label='AlphaEvolve Triangle Scores',
    color='skyblue',
    marker = 'o',
    linestyle='-',
    linewidth=2,
)



plt.plot(
    xs,
    np.array(keich_triangle_shapely_scores[:-3]),
    color='red',
    marker='o',
    label='Keich Construction for Triangles',
)


# Add labels and title
plt.grid()
plt.legend()
plt.xlabel('Number of Points', fontsize=15)
plt.ylabel('Total Union Area', fontsize=15)

In [ ]:
# @title Area graphs (parallelograms)


xs = 2 ** np.arange(1, 8)

plt.figure(figsize=(12, 8))  # Adjust figure size as needed

# Plot Function 1
plt.plot(
    xs,
    np.array(ae_parallelogram_scores),
    label='AlphaEvolve Triangle Scores',
    color='skyblue',
    marker = 'o',
    linestyle='-',
    linewidth=2,
)

# Plot Function 1
# plt.plot(
#     xs,
#     0.5 / np.log(xs),
#     label='0.48 * 1 / Log N',
#     color='orange',
#     linestyle='-',
#     linewidth=2,
# )


def keich_lb_fn(n: int):
  h = np.arange(2, n)
  res = 1 + 3 * (n - 1) / (2 * n) + np.sum(2 * (n - h) / (h * n))
  return 1 / res


klb_vals = [keich_lb_fn(n) for n in xs]


plt.plot(
    xs,
    np.array(list(keich_parallelogram_scores.values())[:-3]), #* np.log(2 ** np.arange(1, 10)),
    color='red',
    marker='o',
    label='Keich Construction for Parallelograms',
)

# plt.plot(
#     xs,
#     klb_vals,
#     label='Keich Lower Bound',
#     color='green',
#     linestyle='-',
#     linewidth=2,
# )


# Add labels and title
plt.grid()
plt.legend()
plt.xlabel('Number of Points', fontsize=15)
plt.ylabel('Total Union Area', fontsize=15)

## Generalization from Keich

**Prompts for generalization with Keich construction hint**

Act as a research mathematician, software developer and optimization specialist
specializing in constructing lists of floats with certain extremal properties.

GOAL:
Your task is to find a general algorithm which, for a given
natural number n > 1, produces a python list x = (x_i)_(i=1)^n consisting of
n positive floats ("positions") which optimize the following task.

Let $\delta = 1 / n$ and define $a_i = i / n$ for $i = 0, \dots, n-1$.
For each $i = 0, \dots, n-1$ let us consider the two-dimensional triangle $R_i$
in the plane given by vertices $(x_i, 0), (x_i + 1 / n, 0), (x_i + a_i, 1)$.

Find x so that the union of the triangles $R_i$ has smallest possible area.

Specifically, the Python function you have to provide is called get_algorithm().
Here is a starting implementation to get you going:

def get_algorithm() -> Callable[[int], list[float]]:
  """Generate optimal points."""

  def get_positions(n: int) -> list[float]:
    best_positions = [x for x in np.linspace(0.0, 0.5, n)]
    return best_positions

  return get_positions

Note that the get_algorithm() function acts as a factory that returns the
final implementation of your algorithm as the function get_positions.
The function get_positions applies your algorithm and outputs the endpoints
[x_1, x_2, ... ,x_n] of the positions. In the body of the factory function
you could test get_positions function or do other preparatory computations.

HINT:
1. Explanation on the starting example: In the case of n being a power of 2,
i.e. n = 2^k consider the following construction. Let us express $a_i := i/n$
in binary as $a_i := \sum_{j=1}^k \epsilon_j 2^{-j}$, and set the position $x_i$
to be $$ x_i := \sum_{j=1}^k \frac{1-j}{n} \epsilon_j 2^{-j}.$$
This construction of x_i might be a good starting point or an inspiration for
further optimization (see the example get_positions function above).
Specifically, the corresponding get_positions function for n being a power of 2
is:

  def get_positions(n: int) -> np.ndarray:
    pos = int(np.log2(n))
    j = np.arange(1, pos + 1)
    x = list()
    for i in range(n):
      eps = np.array(list(str(bin(i))[2:]), dtype=int)
      eps = np.pad(eps, (pos - len(eps), 0), 'constant')
      x.append(np.sum((1 - j) * eps * (2.0 ** -j) / pos))
    return x

2. Try to find simple patterns for the positions.

IMPORTANT CONSTRAINTS:
0. Your algorithm needs to be fast! If get_positions() takes more than 15
seconds to run, you will get a penalty.
1. The function get_positions must be deterministic!
Please keep in mind that if the provided get_positions function is NOT
deterministic (e.g. uses randomness) your algorithm will receive a penalty
during the scoring.
2. Try to keep get_positions concise, short and human-readable.



---------


Act as a research mathematician, software developer and optimization specialist
specializing in constructing lists of floats with certain extremal properties.

GOAL:
Your task is to find a general algorithm which, for a given
natural number n > 1, produces a python list x = (x_i)_(i=1)^n consisting of
n positive floats ("positions") which optimize the following task.

Let us define $a_i = i / n$ for $i = 1, \dots, n$.
For each $i = 0, \dots, n-1$ let us consider the two-dimensional triangle $R_i$
in the plane given by vertices $(x_i, 0), (x_i + 1 / n, 0), (x_i + a_i, 1)$.

Find x so that the union of the triangles $R_i$ has smallest possible area.

Specifically, the Python function you have to provide is called get_positions.
Here is a starting implementation to get you going:

def get_positions(n: int) -> list[float]:
  best_positions = list(np.linspace(0.0, 0.5, n))
  return best_positions

The function get_positions applies your algorithm and outputs the endpoints
[x_1, x_2, ... ,x_n] of the positions.

EVALUATION:
Your proposed construction will be evaluated by an external function that
computes the area of the union. A lower area is better.
You don't have to implement this scoring function; it's already found elsewhere
in the codebase.

HINTS:
1. You will be evaluated against a wide range of small and large values of n, so
you MUST try to find a general solution to the problem. I would strongly
encourage you to try to find a general solution. Your program will be evaluated
on some very large values of n -- try to find the pattern that works for all n.

2. The previous solution provided in this prompt is still not optimal, much
better configurations are possible. The patterns you have to discover are not
hard, you can definitely improve it, it is not beyond your capabilities. DO NOT
go for the same solution as the previous one. Always try to find a better
pattern, don't be scared of the difficult sounding problem, once you see the
solution you'll realise it wasn't hard at all. Good luck!

IMPORTANT CONSTRAINTS:
0. Your algorithm needs to be fast! If get_positions() takes more than 15
seconds to run, you will get a penalty.
1. The function get_positions must be deterministic!
Please keep in mind that if the provided get_positions function is NOT
deterministic (e.g. uses randomness) your algorithm will receive a penalty
during the scoring.
2. Try to keep get_positions concise, short and human-readable.

**Generalization prompt with hints for Keich's construction and jumps**

Act as a research mathematician, software developer and optimization specialist
specializing in constructing lists of floats with certain extremal properties.

GOAL:
Your task is to find a general algorithm which, for a given
natural number n > 1, produces a python list x = (x_i)_(i=1)^n consisting of
n positive floats ("positions") which optimize the following task.

Let $\delta = 1 / n$ and define $a_i = i / n$ for $i = 0, \dots, n-1$.
For each $i = 0, \dots, n-1$ let us consider the two-dimensional triangle $R_i$
in the plane given by vertices $(x_i, 0), (x_i + 1 / n, 0), (x_i + a_i, 1)$.

Find x so that the union of the triangles $R_i$ has smallest possible area.

Specifically, the Python function you have to provide is called get_algorithm().
Here is a starting implementation to get you going:

def get_algorithm() -> Callable[[int], list[float]]:
  """Generate optimal points."""

  def get_positions(n: int) -> list[float]:
    best_positions = [x for x in np.linspace(0.0, 0.5, n)]
    return best_positions

  return get_positions

Note that the get_algorithm() function acts as a factory that returns the
final implementation of your algorithm as the function get_positions.
The function get_positions applies your algorithm and outputs the endpoints
[x_1, x_2, ... ,x_n] of the positions. In the body of the factory function
you could test get_positions function or do other preparatory computations.

HINT:
1. Explanation on the starting example: In the case of n being a power of 2,
i.e. n = 2^k consider the following construction. Let us express $a_i := i/n$
in binary as $a_i := \sum_{j=1}^k \epsilon_j 2^{-j}$, and set the position $x_i$
to be $$ x_i := \sum_{j=1}^k \frac{1-j}{n} \epsilon_j 2^{-j}.$$
This construction of x_i might be a good starting point or an inspiration for
further optimization (see the example get_positions function above).
Specifically, the corresponding get_positions function for n being a power of 2
is:

  def get_positions(n: int) -> np.ndarray:
    pos = int(np.log2(n))
    j = np.arange(1, pos + 1)
    x = list()
    for i in range(n):
      eps = np.array(list(str(bin(i))[2:]), dtype=int)
      eps = np.pad(eps, (pos - len(eps), 0), 'constant')
      x.append(np.sum((1 - j) * eps * (2.0 ** -j) / pos))
    return x

2. Can you make the above pattern work well for non-powers of 2? More
specifically, if n = 2^k, when applied to n + 1 the above construction adds a
triangle that's way too far from the bulk of the other triangles.
If we see the total triangle area as a function of n, this introduces a jump
between n and n+1 - find a way to improve the construction further and avoid
that.

IMPORTANT CONSTRAINTS:
0. Your algorithm needs to be fast!!! If get_positions() takes more than 15
seconds to run, you will get a penalty.
1. The function get_positions must be deterministic!
Please keep in mind that if the provided get_positions function is NOT
deterministic (e.g. uses randomness) your algorithm will receive a penalty
during the scoring.
2. Try to keep get_positions concise, short and human-readable.

In [ ]:
# @title Programs obtained by AlphaEvolve

def get_algorithm() -> Callable[[int], list[float]]:
  """Generate optimal points."""

  def get_positions(n: int) -> list[float]:
    k = (n - 1).bit_length()

    # j_indices represents j from 1 to k, as used in the hint's summation formula.
    j_indices = np.arange(1, k + 1)

    # Tuned constant for coefficients.
    # The value of ALPHA is dynamically adjusted based on the parity of k (bit length of n),
    # introducing a subtle, context-dependent modification to the weighting of bits.
    # This deviation from a fixed ALPHA=0.0 aims to optimize point distribution
    # for specific number-of-bit regimes, which might lead to better packing
    # and reduced union area. The constant 0.08 was selected for its small impact,
    # making it a 'teeny-tiny' but 'crazy' adjustment, perturbing ALPHA only when k is odd.
    ALPHA = (k % 2) * 0.08

    # Pre-calculate the coefficients for the summation.
    # This corresponds to (phi_minus_one - j) * (2.0 ** -j) / log2(n).
    # Using np.log2(n) for smoother scaling than integer k for non-power-of-2 n values.
    # Modified denominator based on randomized exploration, adding 1.0 for improved scaling.
    # The term (np.log2(n) + 1.0) ensures denominator is always >= 1.0 (since n > 1 implies log2(n) > 0).
    # Modified denominator constant to increase point clustering and potentially reduce union area.
    # The value 1.1 was chosen based on an intuition that a slightly higher scaling
    # factor would lead to better packing of the triangles' bases, thereby minimizing their union area.
    # The constant 1.3 is a further refined value based on continued exploration,
    # aiming for even tighter packing of the positions compared to 1.2,
    # which empirically leads to a smaller overall union area of the triangles.
    coeffs = (ALPHA - j_indices) * (2.0 ** -j_indices) / (np.log2(n) + 1.3)

    # Generate all binary digits for all i in a vectorized manner.
    # i_values is a column vector of integers from 0 to n-1.
    i_values = np.arange(n).reshape(-1, 1)

    # shifts determines which bit position to extract for each j_index.
    # For j=1 (MSB), we extract bit k-1; for j=k (LSB), we extract bit 0.
    shifts = k - j_indices

    # eps_matrix (n, k) where eps_matrix[row, col] is the bit corresponding to j_indices[col] for integer row.
    eps_matrix = (i_values >> shifts) & 1

    # Calculate raw x_values using a single matrix multiplication (vectorized summation).
    # This is equivalent to sum(coeffs[j] * eps_matrix[i, j]) for each i.
    raw_x_values = eps_matrix @ coeffs

    # Ensure all positions are positive.
    min_raw_x = np.min(raw_x_values)
    offset = np.maximum(0.0, -min_raw_x) + 1e-9

    # Apply offset and convert the NumPy array to a Python list.
    final_positions = (raw_x_values + offset).tolist()

    return final_positions

  return get_positions



def get_algorithm() -> Callable[[int], list[float]]:
  """Generate optimal points."""

  def get_positions(n: int) -> list[float]:
    """
    This algorithm is a generalization of a construction for n=2^k.
    The construction is inspired by low-discrepancy sequences (van der Corput).
    For a given n, we determine the number of "bits" k needed, as k = (n-1).bit_length(),
    which is equivalent to ceil(log2(n)).
    For each i in 0..n-1, we compute the first k bits of the binary expansion
    of the fraction i/n. Let these bits be eps_j.
    The position x_i is then computed as a weighted sum over these bits,
    where the weights are designed to create good overlap properties.
    Specifically, x_i = (1/k) * sum_{j=1..k} (1-j) * eps_{i,j} * 2^{-j}.
    This formula gives more weight to the less significant bits of i/n,
    creating a kind of 'bit-reversal' effect which tends to cluster the triangles
    effectively, thus reducing the total area of their union.
    The final list of positions is shifted to ensure all values are non-negative.
    """
    if n <= 1:
      return [0.0] * n

    k = (n - 1).bit_length()

    # Get binary expansions of i/n for i=0..n-1.
    # `eps` is a (n, k) matrix of bits.
    i_over_n = np.arange(n, dtype=np.float64) / n
    eps = np.zeros((n, k))

    f = i_over_n.copy()
    for j in range(k):
      f *= 2
      bits = np.floor(f)
      eps[:, j] = bits
      f -= bits

    j_vals = np.arange(1, k + 1)
    term_coeffs = (1 - j_vals) / k
    powers_of_2 = 2.0 ** (-j_vals)

    # Calculate x_i for all i using matrix multiplication.
    x = eps @ (term_coeffs * powers_of_2)

    # Shift to make all positions non-negative.
    x -= np.min(x)

    return x.tolist()

  return get_positions



def get_algorithm() -> Callable[[int], list[float]]:
  """Generate optimal points."""

  def get_positions(n: int) -> list[float]:
    """
    This algorithm generates positions x_i for n triangles, inspired by low-discrepancy sequences
    like the van der Corput sequence.
    For each i in 0..n-1, the algorithm computes the first k bits of the binary expansion
    of the fraction i/n, where k is determined by n.
    The position x_i is then computed as a weighted sum over these bits:
    x_i = sum_{j=1..k} C_j * eps_{i,j} * 2^{-j}.
    The coefficients C_j are dynamically calculated based on 'j' (bit significance), 'k' (total bits),
    and 'n's position within its power-of-2 interval (`normalized_n`).
    This adaptive coefficient generation aims to effectively cluster the triangles and
    smooth area transitions, especially for 'n' values that are not powers of 2.
    The formula uses a combination of hyperbolic and exponential terms to shape the bit
    weights, along with a sinusoidal perturbation to refine overlaps.
    The final list of positions is shifted to ensure all values are non-negative.
    """
    if n <= 1:
      return [0.0] * n

    # Increase k by 1 to capture one more bit of precision for i/n's binary expansion.
    # This aims to smooth out the x_i values, especially for n not a power of 2,
    # and mitigate the 'jump' in area as n crosses power-of-2 boundaries.
    k = (n - 1).bit_length() + 1

    # Get binary expansions of i/n for i=0..n-1.
    # `eps` is a (n, k) matrix of bits.
    i_over_n = np.arange(n, dtype=np.float64) / n
    eps = np.zeros((n, k))

    f = i_over_n.copy()
    for j in range(k):
      f *= 2
      bits = np.floor(f)
      eps[:, j] = bits
      f -= bits

    j_vals = np.arange(1, k + 1)

    # Initialize constant base values for coefficients, formerly perturbed by n_oscillation_factor.
    # Introduce small, n-dependent oscillations to A_offset and B_decay for randomized exploration.
    # Enhanced multi-frequency modulation for A_offset and B_decay
    A_offset = 0.05 + 0.015 * np.sin(n * 0.07) + 0.01 * np.cos(n * 0.12)
    B_decay = 2.0 + 0.3 * np.cos(n * 0.04) + 0.05 * np.sin(n * 0.09)

    # Calculate n's normalized position within its current power-of-2 interval.
    # This helps in smoothing transitions across power-of-2 boundaries.
    k_pow2_floor = int(np.log2(n)) # E.g., for n=3, log2(3)=1.58, int=1. For n=4, log2(4)=2, int=2.
    n_lo_boundary = 2 ** k_pow2_floor
    n_hi_boundary = 2 ** (k_pow2_floor + 1)

    # normalized_n goes from 0 to almost 1 as n progresses from 2^k_floor to 2^(k_floor+1)-1.
    # normalized_n goes from 0 to almost 1 as n progresses from 2^k_floor to 2^(k_floor+1)-1.
    # Modifying normalized_n to never be exactly 0 or 1.
    # This ensures that the sinusoidal components do not completely vanish at power-of-2 boundaries,
    # thereby smoothing the transitions and reducing "jumps" in total area.
    normalized_n = (n - n_lo_boundary + 0.5) / (n_hi_boundary - n_lo_boundary + 1.0)


    # Hyperbolic 'j' warping for base coefficients.
    # This non-linear transformation applies a tanh function to 'j_vals',
    # effectively compressing higher 'j' values and stretching lower 'j' values.
    # This allows for finer control over the influence of more significant bits,
    # and provides a smoother taper for less significant bits, adapting to 'n'
    # for potentially better overall area reduction, especially for non-powers of 2.
    # The 'j_warp_scale' parameter makes the warping 'n'-adaptive.
    # It is perturbed by n_oscillation_factor to explore more dynamics in j-warping.
    j_warp_scale = k / (1.0 + 0.75 * normalized_n) # Adjusts sensitivity based on n's position within power-of-2 interval.
    # Dynamic Bit-Weight Profile Shift:
    # Introduce a small, n-dependent additive offset to 'j_vals' before applying
    # non-linear transformations (tanh, exp). This subtly shifts the effective
    # 'center' of the bit-weight profile, allowing finer tuning of bit significance.
    # The offset magnitude is controlled by n_oscillation_factor and weighted by
    # a cosine of normalized_n, making it more pronounced near power-of-2 boundaries.
    # Introduce n-dependent perturbation to the amplitude of j_adaptive_offset.
    # Enhanced modulation for j_adaptive_offset_base_amplitude
    j_adaptive_offset_base_amplitude = 0.12 + 0.03 * np.sin(n * 0.095) - 0.01 * np.cos(n * 0.05)
    j_adaptive_offset = j_adaptive_offset_base_amplitude * np.cos(normalized_n * np.pi)

    # Apply the adaptive offset to j_vals for the hyperbolic warping.
    j_warped = k * np.tanh((j_vals + j_adaptive_offset) / j_warp_scale)

    # Introduce a 'j-offset' to the base coefficients.
    # This offset is strongest when n is roughly halfway between powers of 2 (sin(pi*0.5)=1),
    # and decays exponentially for higher 'j' (less significant bits).
    # The decay rate is also 'n'-dependent for smoother transitions.
    B_decay_n = B_decay * (1.0 + 0.5 * normalized_n) # Make decay rate n-dependent
    # Apply the adaptive offset to j_vals for the exponential decay in j_offset_n.
    j_offset_n = A_offset * np.sin(normalized_n * np.pi) * np.exp(-(j_vals + j_adaptive_offset) / B_decay_n)

    # The base coefficients now incorporate the hyperbolic warping and n-dependent offset.
    # The initial constant (0.35) is also perturbed by n_oscillation_factor.
    # Perturb base_coeffs_initial_const with an n-dependent oscillation for further exploration.
    # Enhanced modulation for base_coeffs_initial_const
    base_coeffs_initial_const = 0.35 + 0.02 * np.sin(n * 0.085) + 0.015 * np.cos(n * 0.13)
    base_coeffs = (base_coeffs_initial_const - (j_warped + j_offset_n))

    # Add a sinusoidal perturbation to the base coefficients (from original code).
    # This aims to smooth the area function across different 'n', especially
    # near power-of-2 boundaries, and find more optimal overlaps.
    # Chaotic phase modulation for amplitude and frequency modulators.
    # Instead of fixed magic numbers, use irrational multiples of 'n' to generate
    # more chaotic (yet deterministic) and 'n'-sensitive phases.
    # This helps in preventing 'stuck' periodicities and improves smoothing
    # across diverse 'n' values, especially near power-of-2 boundaries.
    # The use of 1.6180339887 (Golden Ratio) and 2.71828 (e) introduces
    # non-linear and non-periodic influences.
    # Simplified sinusoidal perturbation with fixed modulators and n-dependent dampening.
    # Replacing 'chaotic' modulators with fixed values aims to reduce erratic behavior
    # and promote smoother transitions across 'n', especially near power-of-2 boundaries.
    fixed_amplitude_modulator = -0.15 # Chosen from original range [-0.4, -0.1]
    fixed_frequency_modulator = 0.05  # Chosen from original range [0.02, 0.08]

    # Perturb fixed_amplitude_modulator and fixed_frequency_modulator using n-dependent oscillations.
    # This introduces dynamic 'chaotic' elements to the sinusoidal perturbation's properties,
    # allowing for more adaptive and fine-tuned overlaps.
    fixed_amplitude_modulator_adaptive = -0.15 + 0.04 * np.cos(n * 0.11)
    fixed_frequency_modulator_adaptive = 0.05 + 0.015 * np.sin(n * 0.06)

    # The core phase term `j_vals * (n + j_vals / k)` is retained for its 'chirped' frequency effect.
    sin_perturbation = fixed_amplitude_modulator_adaptive * np.sin(j_vals * (n + j_vals / k) * fixed_frequency_modulator_adaptive * np.pi)

    # Introduce a smooth, n-dependent dampening factor for the sinusoidal perturbation.
    # This factor reduces the influence of the perturbation when 'n' is further from
    # a power of 2 (normalized_n closer to 0.5), allowing other n-adaptive terms
    # (like j_offset_n and j_warped) to dominate, thereby smoothing transitions.
    # When 'n' is near a power of 2 (normalized_n near 0 or 1), this dampening_factor
    # is close to 1, allowing the perturbation to have full effect.
    # Modified dampening factor with a wobble for richer n-dependence, especially around boundaries.
    # A combination of cosine and sine terms provides more flexibility in shaping the dampening profile.
    dampening_factor = (0.5 + 0.5 * np.cos(normalized_n * np.pi)) * (1.0 + 0.1 * np.sin(normalized_n * 2 * np.pi))

    term_coeffs = (base_coeffs + dampening_factor * sin_perturbation) / k
    powers_of_2 = 2.0 ** (-j_vals)

    # Calculate x_i for all i using matrix multiplication.
    x = eps @ (term_coeffs * powers_of_2)

    # Shift to make all positions positive.
    # Add a small epsilon to ensure strict positivity as required by the problem.
    x = x - np.min(x) + sys.float_info.epsilon

    return x.tolist()

  return get_positions